# 1.4 Sequencing Antibiotic by Shattering Them into Pieces

## Introduction to mass spectrometry
[NRPs](https://en.wikipedia.org/wiki/Nonribosomal_peptide)는 안타깝게도 ribosome을 제거해도 생성이된다. 즉, 중심이론의 예외에 해당한다. 게다가 NRPs 는 선형이 아니라 cyclic 하기 때문에 sequencing하는 것이 더 어렵다. 이를 해결하기 위해 **Mass spectrometer** 를 사용한다. Mass spectrometer는 원자의 무게를 재는 저울이다. 무게를 재는 단위로는 **Dalton** 을 사용하는데, 1 Da(Dalton) 은 양성자(proton)나 중성자(neutron) 한개의 무개를 의미한다. 예를 들면 C2H3ON은 2·12 + 3·1 + 1·16 + 1·14 = 57 Da 가 된다. 실제로 원자의 무게는 정수로 정확히 떨어지지는 않지만 단순하게 하기위해 정수만을 사용한다.

NRPs sequencing을 어렵게 하는 것은 NRPs가 linear 하지 않고 cyclic 하다는 점이다. 즉, ABC 는 CAB,BCA 와 같다. mass spectrometer는 NRPs 를 sequencing 하기 위해 이 cyclic한 peptide를 잘게 쪼갠다. 그리고, 이렇게 쪼개진 peptide의 집합을 **Experimental Spectrum** 이라고 한다.

여기서 목적은 원자들의 무게와 mass spectrometer가 쪼개놓은 조각들을 이용하여 peptide를 sequencing 하는 것이다.

## The Cyclopeptide Sequencing Problem

### 원문
For now, we will assume for simplicity that the mass spectrometer breaks the copies of a cyclic peptide at every possible two bonds, so that the resulting experimental spectrum contains the masses of all possible linear fragments of the peptide, which are called subpeptides. For example, the cyclic peptide NQEL has 12 subpeptides: N, Q, E, L, NQ, QE, EL, LN, NQE, QEL, ELN, and LNQ. We will also assume that subpeptides may occur more than once if an amino acid occurs multiple times in the peptide (e.g., ELEL also has 12 subpeptides: E, L, E, L, EL, LE, EL, LE, ELE, LEL, ELE, and LEL.

### 자의적 해석
ELEL을 잘라놓으면 E, L, E, L, EL, LE, EL, LE, ELE, LEL, ELE, LEL 와 같이 쪼개진다. 즉, cyclic 한 peptide를 1개씩 쪼개놓을 수도 있고, 2개씩 쪼개 놓을 수도 있고, 3개씩 쪼개놓을 수도 있다는 것이다.(원래의 peptide는 존재하지 않게된다.)

### Problme

**Excercise Break**: How many subpeptides does a cyclic peptide of length n have?

어떤 peptide의 길이가 주어졌을 때, 그 peptide가 쪼개질 수 있는 가지수는 몇가지인가?

In [19]:
import toolz as tz
from toolz.curried import *
from operator import *

In [20]:
input_data = {
    "sample": {
        "n": 31315
    },
    "test": {
        "n": int(open("data/week3/dataset_98_3.txt", "r").read().strip())
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

cur_data["n"] * (cur_data["n"] - 1)

291606852

**Theoretical Spectrum(Cyclospectrun(Peptide))** 는 Subpeptide의 무게들의 집합이다. 이 집합은 0 부터 전체 peptide 무게까지 포함하며, 작은 것부터 큰것까지 무게 순서대로 중복을 포함하는 집합이다.

**Generating Theoretical Spectrum Problem** : Generate the theoretical spectrum of cyclic peptide

**input** : an amino acid string Peptide
**output** : Cyclospectrum(peptide)

## 1.7 CS: Generating the Theoretical Spectrum of a Peptide
subpeptide의 무게는 전체 peptide의 누적 무게에서 subpeptide의 시작 amino acid 바로 이전 값까지의 누적값 을 subpeptide의 끝 amino acid 값까지의 누적값을 뺀 값과 같다.
```
LinearSpectrum(Peptide, Alphabet, AminoAcidMass)
    PrefixMass(0) ← 0
    for i ← 1 to |Peptide|
        for every symbol s in Alphabet
            if s = i-th amino acid in Peptide
                PrefixMass(i) ← PrefixMass(i − 1) + AminoAcidMass[s]
    LinearSpectrum ← a list consisting of the single integer 0
    for i ← 0 to |Peptide| − 1
        for j ← i + 1 to |Peptide|
            add PrefixMass(j) − PrefixMass(i) to LinearSpectrum
    return sorted list LinearSpectrum
```

**Code Challenge**

**Input**: An amino acid string peptide
**Output**: The linear spectrum of Peptide

In [45]:
input_data = {
    "sample": {
        "peptide": "LEQN",
    },
    "test": {
        "peptide": open("data/week3/dataset_98_4.txt").read().strip()
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

mass_table = {}
with open("data/week3/integer_mass_table.txt", "r") as f:
    for line in f:
        k, v = line.strip().split(" ")
        mass_table[k] = int(v)

def linear_spectrum(peptide, amino_acid_mass):
    mass_spectrum =[0, *accumulate(add, [amino_acid_mass[aa] for aa in peptide])]
    cyclic_mass_spectrum = mass_spectrum.copy()
    peptide_mess = last(mass_spectrum)
    for i in range(1, len(peptide)):
        for j in range(i + 1, len(peptide) + 1):
            cyclic_mass_spectrum.append(mass_spectrum[j] - mass_spectrum[i])
            if i > 0 and j < len(peptide):
                cyclic_mass_spectrum.append(peptide_mess - (mass_spectrum[j] - mass_spectrum[i]))

    return sorted(cyclic_mass_spectrum)
print(*linear_spectrum(cur_data["peptide"], mass_table))

0 87 97 97 99 103 113 115 128 128 128 131 131 163 196 196 218 218 218 225 225 234 241 243 256 276 293 294 321 324 324 338 346 349 349 353 371 381 404 407 421 421 436 437 452 452 468 474 477 494 501 512 534 535 549 549 564 567 567 571 600 605 615 622 625 632 662 664 670 677 692 695 697 702 719 728 730 731 753 767 789 790 792 801 818 823 825 828 843 850 856 858 888 895 898 905 915 920 949 953 953 956 971 971 985 986 1008 1019 1026 1043 1046 1052 1068 1068 1083 1084 1099 1099 1113 1116 1139 1149 1167 1171 1171 1174 1182 1196 1196 1199 1226 1227 1244 1264 1277 1279 1286 1295 1295 1302 1302 1302 1324 1324 1357 1389 1389 1392 1392 1392 1405 1407 1417 1421 1423 1423 1433 1520
